In [1]:
import os, os.path
from PIL import Image
import requests
from io import BytesIO
import xml.etree.ElementTree as ET
import tqdm
import pandas as pd
import numpy as np
import imagehash

SCRAPE_DIR = '.scrape_output'

### Steps to fetch the xml files
1. Install `pip install oaiharvest`
2. Create a folder in this repo called '.scrape_output'
3. `cd .scrape_output`
4. Run the following command: `oai-harvest --set=Beelddocument --metadataPrefix=ese https://denbosch.hosting.deventit.net/atlantispubliek/oai.axd` (This may take a while)

### Test code to get the format of the xml file

In [2]:
tree = ET.parse('.scrape_output/14433440.ese.xml')
root = tree.getroot()

items = []

for child in root:
    prefix, has_namespace, postfix = child.tag.partition('}')
    items.append({
        'key': postfix,
        'value': child.text
    })

pd.DataFrame(items)

,key,value
0,title,Heien van de eerste paal voor de woningen voor...
1,creator,"Janssens, Felix"
2,subject,Exterieur
3,subject,Woningbouw
4,subject,Nieuwbouw
5,coverage,'s-Hertogenbosch;Schaarhuispad
6,publisher,Stadsarchief 's-Hertogenbosch
7,publisher,http://www.stadsarchief.nl/
8,date,10/3/1987
9,type,Foto


### Create a list of the XML files

In [3]:
image_xml_files = [SCRAPE_DIR + '/' + name for name in os.listdir(SCRAPE_DIR)]
image_xml_files[:3]

['.scrape_output/104915350.ese.xml',
 '.scrape_output/104915448.ese.xml',
 '.scrape_output/104915560.ese.xml']

#### Loop through the images and transform them into a dictionaries

In [311]:
images = []

# xml namespace
ns = {
    'dc': 'http://purl.org/dc/elements/1.1/',
    'europeana': 'http://www.europeana.eu/schemas/ese/',
    'dcterms': 'http://purl.org/dc/terms/'
}

# No image hash
no_img = '00ff6767777f8300'

for image_xml in image_xml_files:
    # Parse as xml
    tree = ET.parse(image_xml)
    
    # Get the root of the tree
    root = tree.getroot()
    
    # Dictionary to store the image attributes
    image_attribs = {}
    
    # Image url
    image_url = root.find('europeana:object', ns).text
    
    # Check if the image exists
#     response = requests.get(image_url)
#     image = Image.open(BytesIO(response.content))
#     hasj = str(imagehash.average_hash(image))
    
#     # If there is no image
#     if hasj == no_img:
#         continue

#     # Size of the image
#     image_attribs['width'], image_attribs['height'] = image.size
    
    # Image url
    image_attribs['url'] = image_url
    
    # Get the title
    image_attribs['title'] = root.find('dc:title', ns).text.strip()
    
    # Location
    coverage = root.find('dc:coverage', ns)
    
    if coverage != None:
        coverage = coverage.text.split(';')
        
        # Don't store the image if it is not located in Den Bosch
        if coverage[0] != "'s-Hertogenbosch":
            continue
            
        street = coverage[1]
        image_attribs['street'] = street
            
    # Date
    if root.find('dc:date', ns).text == None or len(root.find('dc:date', ns).text[-4:]) < 4 or '/' in root.find('dc:date', ns).text[-4:]:
        image_attribs['year'] = None
    else: 
        image_attribs['year'] = root.find('dc:date', ns).text[-4:]
        
    # Medium
    medium = root.find('dcterms:medium', ns)
    
    # Make sure the image is a photo
    if medium == None:
        continue
    elif medium.text != 'Foto':
        continue
    
    # Add to list
    images.append(image_attribs)

In [315]:
df = pd.DataFrame(images)
df.head()

,street,title,url,year
0,Muntelplein,Arbeiderswoningen van woningbouwvereniging mgr...,http://denbosch.hosting.deventit.net/HttpHandl...,1930
1,,Arbeiderswoningen van woningbouwvereniging mgr...,http://denbosch.hosting.deventit.net/HttpHandl...,1930
2,,Arbeiderswoningen van woningbouwvereniging mgr...,http://denbosch.hosting.deventit.net/HttpHandl...,1930
3,,Arbeiderswoningen van woningbouwvereniging mgr...,http://denbosch.hosting.deventit.net/HttpHandl...,1930
4,Citadellaan,Woningen ontworpen door architect Jan van den ...,http://denbosch.hosting.deventit.net/HttpHandl...,1930


In [40]:
# Import dataframe into MySQL
import sqlalchemy
database_username = 'remote'
database_password = 'EtrPCEc0jt'
database_ip       = '165.22.199.122'
database_name     = 'scavenger'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))
df.to_sql(con=database_connection, name='images', if_exists='replace')

In [309]:
import requests
import json

# HERE app_id & app_code
APP_ID = 'oidKwC98hLcIQBaboIwS'
APP_CODE = 'G6gKSvk45y4fF14hwpIeYg'

# Function that collects multiple location-coordinates of a single street
def get_location(streetname):
    '''Returns list of latitude and longitude coordinates for a given street'''
    
    if streetname == '':
        return None
    
    url = 'https://geocoder.api.here.com/6.2/geocode.json?app_id=' + APP_ID + '&app_code=' + APP_CODE + '&searchtext='

    r = requests.get(url + streetname + ", 's-Hertogenbosch").json()
    
    if r['Response']['View'] != []:
        
        loc_data = r['Response']['View'][0]['Result'][0]['Location']
        
        #DisplayPosition
        loc1 = loc_data['DisplayPosition']
    
        #NavigationPosition
        loc2 = loc_data['NavigationPosition'][0]
    
        #MapView
        loc3A = loc_data['MapView']['TopLeft']
        
        loc3B = loc_data['MapView']['BottomRight']
        
        #Avg coordinates b/w Mapview TopLeft & Mapview BottomRight
        loc4 = ((loc3A['Latitude'] + loc3B['Latitude']) / 2, (loc3A['Longitude'] + loc3B['Longitude']) / 2 )
        
        return [loc1['Latitude'], loc1['Longitude'], 
                loc2['Latitude'], loc2['Longitude'], 
                loc3A['Latitude'], loc3A['Longitude'],
                loc3B['Latitude'], loc3B['Longitude'], 
                loc4[0], loc4[1]]
    
    else:
        return None

In [413]:
# All streetnames present in the dataframe
street_names = list(df['street'].unique())

# Remove from street_names list: nan value at index location 66, '' at index location 1, and 'onbekend' at index location 537
del street_names[66]
del street_names[1]
del street_names[537]

# Generate dictionary with street location and all corresponding locations
street_locations = {street : get_location(street) for street in street_names}

In [414]:
# Find all streets that API did NOT find location coordinates for
failed_location = [street[0] for street in street_locations.items() if street[1] == None]

In [416]:
# Remove streets from street_locations dictionary that do not have any coordinates
[street_locations.pop(street) for street in failed_location]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [417]:
# Column order of coordinates (dis = display location, nav = navigation location, map_top = mapview topleft, map_bot = mapview bottomright, map_avg = mapview average)
column_order = ['dis_lat', 'dis_long', 'nav_lat', 'nav_long', 'map_top_lat', 'map_top_long', 'map_bot_lat', 'map_bot_long', 'map_avg_lat', 'map_avg_long']

In [433]:
# Create dataframe with streets as rows and coordinates as columns
loc_df = pd.DataFrame.from_dict(street_locations, orient='index', columns=column_order)
loc_df.reset_index(level=0, inplace=True)
loc_df.rename(columns={'index':'street'}, inplace=True)
loc_df.head()

,street,dis_lat,dis_long,nav_lat,nav_long,map_top_lat,map_top_long,map_bot_lat,map_bot_long,map_avg_lat,map_avg_long
0,Muntelplein,51.69426,5.30766,51.69426,5.30766,51.69467,5.30671,51.69420,5.30770,51.694435,5.307205
1,Citadellaan,51.69688,5.30403,51.69688,5.30403,51.69793,5.30368,51.69448,5.30569,51.696205,5.304685
2,Brugplein,51.69439,5.29895,51.69439,5.29895,51.69458,5.29857,51.69418,5.29935,51.694380,5.298960
3,Uilenburg,51.68760,5.29959,51.68760,5.29959,51.68888,5.29857,51.68687,5.29995,51.687875,5.299260
4,Hinthamerstraat,51.68902,5.30550,51.68902,5.30550,51.68911,5.30444,51.68892,5.30657,51.689015,5.305505


In [434]:
# Import dataframe into MySQL
import sqlalchemy
database_username = 'remote'
database_password = 'EtrPCEc0jt'
database_ip       = '165.22.199.122'
database_name     = 'scavenger'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))
loc_df.to_sql(con=database_connection, name='locations', if_exists='replace')